In [1]:
import requests
#from api_keys import weather_api_key
import pandas as pd
import numpy as np
import datetime as dt
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
pd.set_option('max_colwidth', 400)

In [57]:
# Light Pollution Data Base

# Used csv file to get data from 2022, tbd if we go this way or with a json file.
lightpol = pd.read_csv("Resources/GaN2022.csv")

# Create new state column and drop entries without state
lightpol[['Country', 'State']] = lightpol['Country'].str.split(' - ', expand=True)
lightpol.dropna(subset=['State'], inplace=True)

# Clean LimitingMag data to only have > 0 values and dropping unwanted columns
lp_clean = lightpol.loc[(lightpol['LimitingMag']>=0)&
                        (lightpol['Latitude']!=0)&
                        (lightpol['Longitude']!=0)&
                        (lightpol['Country'].str.contains("United States")),
                        ['ObsDateTime','Latitude','Longitude','Elevation','LimitingMag','Constellation','State']]

# Rename Limiting Mag column 

lp_clean = lp_clean.rename(columns={'LimitingMag':'NELM'})


# Change data types

lp_clean.Latitude = lp_clean.Latitude.astype('float')
lp_clean.NELM = lp_clean.NELM.astype('float')

# Save to CSV file to create SQL Database:
lp_clean.to_csv("Resources/lightpollution.csv", encoding='utf8', index=False)

lp_clean.head()

# Should we do something about the ObsDateTime and UTCOffset?

,ObsDateTime,Latitude,Longitude,Elevation,NELM,Constellation,State
7,2022-11-21 20:45:00+00:00,31.334094,-109.521212,1249.21,0.0,Canis Major,Arizona
8,2022-11-21 20:36:00+00:00,31.334086,-109.503952,1258.78,0.0,Canis Major,Arizona
9,2022-11-21 20:32:00+00:00,31.334107,-109.487273,1288.52,0.0,Canis Major,Arizona
10,2022-11-21 20:26:00+00:00,31.334069,-109.470129,1311.26,5.0,Canis Major,Arizona
11,2022-11-21 20:22:00+00:00,31.334055,-109.453368,1340.61,5.0,Canis Major,Arizona


In [58]:
# Lat and lon filter, not considering Alaska

lp_cleanv2 = lp_clean.loc[((lp_clean['Latitude']>=25)&
                         (lp_clean['Latitude']<=50)&
                         (lp_clean['Longitude']>=-125)&
                         (lp_clean['Longitude']<=-65))|
                         ((lp_clean['Latitude']>=18)&
                         (lp_clean['Latitude']<=23)&
                         (lp_clean['Longitude']>=-165)&
                         (lp_clean['Longitude']<=-150)),:]

# Save to CSV file to create SQL Database:
lp_cleanv2.to_csv("Resources/lightpollution_v2.csv", encoding='utf8', index=False)

lp_cleanv2.head()

,ObsDateTime,Latitude,Longitude,Elevation,NELM,Constellation,State
7,2022-11-21 20:45:00+00:00,31.334094,-109.521212,1249.21,0.0,Canis Major,Arizona
8,2022-11-21 20:36:00+00:00,31.334086,-109.503952,1258.78,0.0,Canis Major,Arizona
9,2022-11-21 20:32:00+00:00,31.334107,-109.487273,1288.52,0.0,Canis Major,Arizona
10,2022-11-21 20:26:00+00:00,31.334069,-109.470129,1311.26,5.0,Canis Major,Arizona
11,2022-11-21 20:22:00+00:00,31.334055,-109.453368,1340.61,5.0,Canis Major,Arizona


In [4]:
# Bortle Scale VS Limiting Magnitude Data Base

# Data scraping to get Bortle Scale vs Limiting Magnitude from Wikipedia

browser = Browser('chrome')
url = 'https://en.wikipedia.org/wiki/Bortle_scale'
browser.visit(url)
html = browser.html
browser.quit()
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', class_='wikitable')
headers = [header.get_text().strip() for header in table.find_all('th')]
rows = table.find_all('tr')
tabledata = []
for row in rows:
    columns = row.find_all('td')
    rowdata = [column.get_text().strip() for column in columns]
    tabledata.append(rowdata)

# DataFrame for Bortle - NELM equivalence

bortledf = pd.DataFrame(tabledata, columns=headers)
bortledf = bortledf.iloc[1:6,0:3]
bortledf.head()

,Class,Title,NELM
1,1,Excellentdark-sky site,7.6–8.0
2,2,Typical trulydark site,7.1–7.5
3,3,Rural sky,6.6–7.0
4,4,Brighter rural,6.3–6.5
5,4.5,Semi-Suburban/Transition sky,6.1–6.3


In [5]:
# Separate NELM min and max
bortledf[['Min. NELM','Max. NELM']] = bortledf.NELM.str.split('–', expand=True)
bortledf

,Class,Title,NELM,Min. NELM,Max. NELM
1,1,Excellentdark-sky site,7.6–8.0,7.6,8.0
2,2,Typical trulydark site,7.1–7.5,7.1,7.5
3,3,Rural sky,6.6–7.0,6.6,7.0
4,4,Brighter rural,6.3–6.5,6.3,6.5
5,4.5,Semi-Suburban/Transition sky,6.1–6.3,6.1,6.3


In [64]:
# Change data types and set index
bortledf_clean = bortledf.drop(columns='NELM')
bortledf_clean = bortledf_clean.astype({'Class':float,'Min. NELM':float, 'Max. NELM':float})
#bortledf_clean = bortledf_clean.set_index('Class')

# Save to csv
bortledf_clean.to_csv("Resources/bortle_nelm_eq.csv", encoding='utf8', index=False)

bortledf_clean.head()

,Class,Title,Min. NELM,Max. NELM
1,1.0,Excellentdark-sky site,7.6,8.0
2,2.0,Typical trulydark site,7.1,7.5
3,3.0,Rural sky,6.6,7.0
4,4.0,Brighter rural,6.3,6.5
5,4.5,Semi-Suburban/Transition sky,6.1,6.3


In [33]:
bortledf_clean.dtypes

Class        float64
Title         object
Min. NELM    float64
Max. NELM    float64
dtype: object

In [7]:
# Moon Data

browser = Browser('chrome')
url = 'https://aa.usno.navy.mil/calculated/moon/fraction?year=2022&task=00&tz=5&tz_sign=-1&tz_label=false&submit=Get+Data'
browser.visit(url)
html = browser.html
browser.quit()
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table')
moontable = tables[1]
rows = moontable.find_all('tr')
tabledata2 = []
for row in rows:
    columns = row.find_all('td')
    rowdata = [column.get_text().strip() for column in columns]
    tabledata2.append(rowdata)

In [91]:
lightpol_final['NELM'].loc[7]
list(bortledf_clean['Title'].values)[::-1]


['Semi-Suburban/Transition sky',
 'Brighter rural',
 'Rural sky',
 'Typical trulydark site',
 'Excellentdark-sky site']

In [70]:
# Merging everything into one DataFrame: Bortle Scale
# Establish the bins 
min_nelm = bortledf_clean['Min. NELM'].values.tolist()
min_nelm.append(0)
nelm_bins = sorted(min_nelm)
labels = sorted(bortledf_clean['Title'].values.tolist())
min_nelm
# Create a copy of the light pollution  
lightpol_final = lp_cleanv2.copy()
lightpol_final.head()
# Use `pd.cut` to categorize spending based on the bins.
lightpol_final["Bortle Class"] = pd.cut(lightpol_final["NELM"], 
                                              nelm_bins, labels=labels, 
                                              include_lowest=True)
# Save to csv
lightpol_final.to_csv("Resources/lightpol_final.csv", encoding='utf8', index=False)
lightpol_final.head(10)

,ObsDateTime,Latitude,Longitude,Elevation,NELM,Constellation,State,Bortle Class
7,2022-11-21 20:45:00+00:00,31.334094,-109.521212,1249.21,0.0,Canis Major,Arizona,Brighter rural
8,2022-11-21 20:36:00+00:00,31.334086,-109.503952,1258.78,0.0,Canis Major,Arizona,Brighter rural
9,2022-11-21 20:32:00+00:00,31.334107,-109.487273,1288.52,0.0,Canis Major,Arizona,Brighter rural
10,2022-11-21 20:26:00+00:00,31.334069,-109.470129,1311.26,5.0,Canis Major,Arizona,Brighter rural
11,2022-11-21 20:22:00+00:00,31.334055,-109.453368,1340.61,5.0,Canis Major,Arizona,Brighter rural
12,2022-11-21 20:18:00+00:00,31.334185,-109.419595,1405.68,6.0,Canis Major,Arizona,Brighter rural
13,2022-11-21 20:14:00+00:00,31.334126,-109.419610,1405.75,6.0,Canis Major,Arizona,Brighter rural
14,2022-11-21 20:08:00+00:00,31.334133,-109.402590,1457.63,6.0,Canis Major,Arizona,Brighter rural
15,2022-11-21 20:02:00+00:00,31.334030,-109.385996,1327.25,6.0,Canis Major,Arizona,Brighter rural
16,2022-11-21 19:58:00+00:00,31.334063,-109.369043,1286.75,6.0,Canis Major,Arizona,Brighter rural


In [8]:
# Gets only the data we need
moondf = pd.DataFrame(tabledata2[2:],columns=tabledata2[1])
# Replaces '--' values and changes data types of the rest
moondf = moondf.replace('--', None)
moondf = moondf.astype('float')
moondf = moondf.astype({'Day':int})
#moondf = moondf.set_index('Day')
# Saves into a CSV file
moondf.to_csv("Resources/moon_phases.csv", encoding='utf8', index=False)
# Shows data frame
moondf

,Day,Jan.,Feb.,Mar.,Apr.,May,June,July,Aug.,Sep.,Oct.,Nov.,Dec.
0,1,0.04,0.00,0.03,0.00,0.00,0.03,0.04,0.11,0.23,0.30,0.49,0.57
1,2,0.01,0.01,0.01,0.01,0.02,0.07,0.09,0.18,0.33,0.41,0.61,0.67
2,3,0.00,0.05,0.00,0.04,0.05,0.12,0.15,0.27,0.44,0.52,0.71,0.77
3,4,0.03,0.11,0.03,0.09,0.10,0.19,0.22,0.37,0.55,0.64,0.81,0.85
4,5,0.09,0.19,0.07,0.15,0.17,0.27,0.31,0.47,0.66,0.74,0.89,0.92
5,6,0.16,0.28,0.13,0.23,0.24,0.36,0.41,0.58,0.77,0.84,0.95,0.96
6,7,0.25,0.37,0.21,0.31,0.33,0.46,0.51,0.69,0.86,0.91,0.98,0.99
7,8,0.35,0.47,0.29,0.40,0.42,0.56,0.62,0.79,0.93,0.97,1.00,1.00
8,9,0.45,0.56,0.38,0.49,0.52,0.66,0.73,0.88,0.98,0.99,0.99,0.99
9,10,0.55,0.65,0.48,0.59,0.62,0.76,0.82,0.95,1.00,1.00,0.97,0.96
